In [196]:
import numpy as np
from copy import deepcopy
import random

In [197]:
# Tile wall index values
TILE_WALL_VERTICAL = 0
TILE_WALL_HORIZONTAL = 1
TILE_WALL_CORNER_BOTTOM_LEFT = 2
TILE_WALL_CORNER_BOTTOM_RIGHT = 3
TILE_WALL_CORNER_TOP_RIGHT = 4
TILE_WALL_CORNER_TOP_LEFT = 5
TILE_FLOOR = 6
TILE_DOOR_HORIZONTAL = 7
TILE_DOOR_VERTICAL = 8
TILE_INVALID = -1

In [198]:
#Constants for genetic algorithm parameters
POPULATION_SIZE = 100
MAX_GENERATIONS = 100
MUTATION_RATE = 0.5
NUM_ROOMS = 3 #Target number of rooms
MAP_WIDTH = 10
MAP_HEIGHT = 10

In [199]:
class parameters:
    def __init__(self, population_size=POPULATION_SIZE, max_generations=MAX_GENERATIONS, mutation_rate=MUTATION_RATE,map_width= MAP_WIDTH,map_height=MAP_HEIGHT,num_rooms=NUM_ROOMS):
        self.map_width = map_width
        self.map_height = map_height
        self.population_size = population_size
        self.max_generations = max_generations
        self.mutation_rate = mutation_rate
        self.num_rooms = num_rooms
        self.birth_rate_per_generation = 1
        self.max_number_of_generations = 50
        self.bonus_for_one_door = 2
        self.bonus_for_two_doors = 4
        self.door_positions = generate_door_positions(self.map_width,self.map_height,self.num_rooms)

In [200]:
def generate_door_positions(width,height,num_rooms):
        """
        Generate immutable door positions for the map.
        Doors are restricted to being within N-3 x N-3 to ensure they are further inside the map.
        Vertical doors are accompanied by an additional door at x-1,
        Horizontal doors are accompanied by an additional door at y-1.
        """
        positions = []
        for _ in range(num_rooms - 1):  # Generate `num_rooms - 1` doors
            while True:
                # Generate door positions further inside the map
                x = random.randint(2, width - 3)
                y = random.randint(2, height - 3)
                orientation = random.choice([TILE_DOOR_HORIZONTAL, TILE_DOOR_VERTICAL])  # Random door type

                if orientation == TILE_DOOR_VERTICAL:
                    # Ensure space for the companion door at (x-1, y)
                    if x - 1 >= 1:  # Check bounds
                        positions.append((x, y, TILE_DOOR_VERTICAL))  # Main door
                        positions.append((x - 1, y, TILE_DOOR_VERTICAL))  # Companion door
                        break
                elif orientation == TILE_DOOR_HORIZONTAL:
                    # Ensure space for the companion door at (x, y-1)
                    if y - 1 >= 1:  # Check bounds
                        positions.append((x, y, TILE_DOOR_HORIZONTAL))  # Main door
                        positions.append((x, y - 1, TILE_DOOR_HORIZONTAL))  # Companion door
                        break

        return positions

In [201]:
class LevelGenerationProblem:
    def __init__(self, params):
        self.width = params.map_width  # Map width (N)
        self.height = params.map_height  # Map height (N)
        self.parameters = params
        self.num_rooms = params.num_rooms  # Number of rooms
        self.door_positions = params.door_positions


    def calculate_fitness(self, chromosome):
        """
        Fitness evaluation for a map, returning fitness and a score map.
        """
        map_with_doors = chromosome
        # Initialize fitness and create a score map same shape as the map
        fitness_score = 0
        score_map = np.zeros((self.height, self.width), dtype=int)

        # Iterate through every tile on the map
        for y in range(self.height):
            for x in range(self.width):
                tile_type = map_with_doors[y, x]

                # Skip non-evaluable tiles
                if tile_type not in [
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_VERTICAL,
                    TILE_DOOR_HORIZONTAL,
                    TILE_DOOR_VERTICAL,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_TOP_LEFT,
                ]:
                    continue

                # Evaluate the single tile at (x, y)
                is_valid, tile_score = self.validate_tile(map_with_doors, x, y)

                # Aggregate the tile's contribution to the total fitness
                fitness_score += tile_score

                # Update the score map for the current tile
                score_map[y, x] = tile_score

        valid_wall_map = self.validate_connected_tiles(score_map, map_with_doors)
        #based on valid_wall_map, apply bonuses to scoring.
        updated_score_map = self.bonus_score_for_validated_map(score_map, valid_wall_map)

        # Return total fitness and the aligned score map as a 2D numpy array
        return fitness_score, updated_score_map, valid_wall_map

    def bonus_score_for_validated_map(self, score_map, valid_wall_map):
        # Iterate through every position in the map
        for y in range(score_map.shape[0]):  # Loop over rows
            for x in range(score_map.shape[1]):  # Loop over columns
                # If the tile is valid (indicated by 1 in valid_wall_map), double the score
                if valid_wall_map[y, x] == 1:
                    score_map[y, x] *= self.parameters.bonus_for_one_door
                elif valid_wall_map[y, x] == 2:
                    score_map[y, x] *= self.parameters.bonus_for_two_doors

        # Return the modified score_map
        return score_map


    def validate_tile(self, tile_map, x, y):
        """
        Evaluate a single tile in the given map.
        Returns:
            is_valid (bool): Whether the tile contributes validly to the enclosure.
            tile_score (int): The score of the tile.
        """
        # Ensure the tile lies within valid map bounds
        if not (0 <= x < self.width and 0 <= y < self.height):
            return False, -1, {(x, y): -1}  # Invalid position outside map bounds

        tile = tile_map[y, x]  # Access the tile
        is_valid = True  # By default, assume the tile is valid
        tile_score = 0  # Tile's contribution to fitness

        # Evaluate the tile's score based on its type
        if tile in [TILE_WALL_HORIZONTAL, TILE_DOOR_HORIZONTAL]:
            is_valid, tile_score = self.validate_horizontal_tile(tile_map, x, y)
        elif tile in [TILE_WALL_VERTICAL, TILE_DOOR_VERTICAL]:
            is_valid, tile_score = self.validate_vertical_tile(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_BOTTOM_LEFT:
            is_valid, tile_score = self.validate_bottom_left_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_BOTTOM_RIGHT:
            is_valid, tile_score = self.validate_bottom_right_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_TOP_RIGHT:
            is_valid, tile_score = self.validate_top_right_corner(tile_map, x, y)
        elif tile == TILE_WALL_CORNER_TOP_LEFT:
            is_valid, tile_score = self.validate_top_left_corner(tile_map, x, y)
        elif tile == TILE_FLOOR:  # Assign flat score for floor tiles
            is_valid, tile_score = True, 1
        else:  # Invalid or unknown tile types
            is_valid, tile_score = False, -1

        # Return the results for this single tile
        return is_valid, tile_score



    def validate_horizontal_tile(self, tile_map, x, y):
        """
        Validate a horizontal wall or door tile by considering its left, right,
        and whether its partner door is above or below.
        """
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_INVALID
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_INVALID
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_INVALID
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_INVALID

        # Determine if the door has a partner above or below
        has_partner_above = above == TILE_DOOR_HORIZONTAL
        has_partner_below = below == TILE_DOOR_HORIZONTAL

        # Base validation for left and right
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_BOTTOM_LEFT,
                              TILE_DOOR_HORIZONTAL]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_WALL_CORNER_BOTTOM_RIGHT,
                                TILE_DOOR_HORIZONTAL]

        # Adjust validation based on partner position
        if has_partner_above:
            # Partner is above: Exclude bottom corners
            valid_left = valid_left and left not in [TILE_WALL_CORNER_BOTTOM_LEFT]
            valid_right = valid_right and right not in [TILE_WALL_CORNER_BOTTOM_RIGHT]
        elif has_partner_below:
            # Partner is below: Exclude top corners
            valid_left = valid_left and left not in [TILE_WALL_CORNER_TOP_LEFT]
            valid_right = valid_right and right not in [TILE_WALL_CORNER_TOP_RIGHT]

        is_valid = valid_left and valid_right

        # Adjust scoring
        base_score = 10
        door_bonus = 50 if tile_map[y, x] == TILE_DOOR_HORIZONTAL else 0
        neighbor_door_bonus = 20 if left == TILE_DOOR_HORIZONTAL or right == TILE_DOOR_HORIZONTAL else 0

        score = base_score + door_bonus + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score


    def validate_vertical_tile(self, tile_map, x, y):
        """
        Validate a vertical wall or door tile by checking its top, bottom, left, and right neighbors.
        """
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_INVALID
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_INVALID
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_INVALID
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_INVALID

        # Determine if the door has a partner to the left or right
        has_partner_left = left == TILE_DOOR_VERTICAL
        has_partner_right = right == TILE_DOOR_VERTICAL

        # Base validation for above and below
        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT,
                                TILE_DOOR_VERTICAL]
        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT,
                                TILE_DOOR_VERTICAL]

        # Adjust validation based on partner position
        if has_partner_right:
            # Partner is to the right: Disallow top-left and bottom-left corners
            valid_above = valid_above and above not in [TILE_WALL_CORNER_TOP_LEFT]
            valid_below = valid_below and below not in [TILE_WALL_CORNER_BOTTOM_LEFT]
        elif has_partner_left:
            # Partner is to the left: Disallow top-right and bottom-right corners
            valid_above = valid_above and above not in [TILE_WALL_CORNER_TOP_RIGHT]
            valid_below = valid_below and below not in [TILE_WALL_CORNER_BOTTOM_RIGHT]

        is_valid = valid_above and valid_below

        # Adjust scoring
        base_score = 10
        door_bonus = 50 if tile_map[y, x] == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus = 20 if above == TILE_DOOR_VERTICAL or below == TILE_DOOR_VERTICAL else 0

        score = base_score + door_bonus + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score



    def validate_bottom_left_corner(self, tile_map, x, y):
        """Validate a bottom-left corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_INVALID
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_INVALID

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_VERTICAL]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_above and valid_right
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if above == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if right == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_bottom_right_corner(self, tile_map, x, y):
        """Validate a bottom-right corner based on its neighbors."""
        above = tile_map[y - 1, x] if y - 1 >= 0 else TILE_INVALID
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_INVALID

        valid_above = above in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_VERTICAL]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_above and valid_left
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if above == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if left == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_top_right_corner(self, tile_map, x, y):
        """Validate a top-right corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_INVALID
        left = tile_map[y, x - 1] if x - 1 >= 0 else TILE_INVALID

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_VERTICAL]
        valid_left = left in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_below and valid_left
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if below == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if left == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_top_left_corner(self, tile_map, x, y):
        """Validate a top-left corner based on its neighbors."""
        below = tile_map[y + 1, x] if y + 1 < self.height else TILE_INVALID
        right = tile_map[y, x + 1] if x + 1 < self.width else TILE_INVALID

        valid_below = below in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_VERTICAL]
        valid_right = right in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_HORIZONTAL]

        is_valid = valid_below and valid_right
        base_score = 10
        # Add bonuses for neighboring doors
        neighbor_door_bonus = 20 if below == TILE_DOOR_VERTICAL else 0
        neighbor_door_bonus += 20 if right == TILE_DOOR_HORIZONTAL else 0

        # Calculate final score
        score = base_score + neighbor_door_bonus
        score = score if is_valid else -3

        return is_valid, score

    def validate_connected_tiles(self, score_map, tile_map):
        """
        Validates connected tiles, ensuring walls connect correctly to doors.

        Inputs:
            score_map: Fitness scoring map for penalties/rewards.
            tile_map: Map containing the specific tile type at each (x, y).

        Outputs:
            valid_wall_map: Map showing which tiles are valid, penalized, or rewarded.
        """
        valid_wall_map = np.zeros((self.height, self.width), dtype=int)
        visited = set()  # Track visited tiles to avoid redundant checks

        def is_valid_tile_neighbor(tile_pos, neighbor_pos, tile_map):
            """
            Validates whether `tile_pos` and `neighbor_pos` connect correctly, considering
            both the tile types in `tile_map` and the relative position of the neighbor.
            """
            ty, tx = tile_pos
            ny, nx = neighbor_pos

            # Ensure neighbors are within bounds
            if not (0 <= nx < self.width and 0 <= ny < self.height):
                return False  # Out-of-bounds tiles are invalid neighbors

            # Tile types
            tile_type = tile_map[ty, tx]
            neighbor_type = tile_map[ny, nx]

            # Determine the relative position of the neighbor
            relative_position = (ny - ty, nx - tx)  # (row_diff, col_diff)

            # Check rules for connecting based on the current tile type
            if tile_type == TILE_WALL_VERTICAL:
                # Neighbors above or below
                if relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT,
                                             TILE_DOOR_VERTICAL]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT,
                                             TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_HORIZONTAL:
                # Neighbors to the left or right
                if relative_position == (0, -1):  # Left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_BOTTOM_LEFT,
                                             TILE_DOOR_HORIZONTAL]
                elif relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_WALL_CORNER_BOTTOM_RIGHT,
                                             TILE_DOOR_HORIZONTAL]

            elif tile_type == TILE_WALL_CORNER_TOP_LEFT:
                # Valid neighbors for top-left corner
                if relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_CORNER_TOP_RIGHT:
                # Valid neighbors for top-right corner
                if relative_position == (0, -1):  # left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_CORNER_BOTTOM_LEFT:
                # Valid neighbors for bottom-left corner
                if relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_RIGHT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_WALL_CORNER_BOTTOM_RIGHT:
                # Valid neighbors for bottom-right corner
                if relative_position == (0, -1):  # Left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_DOOR_HORIZONTAL]
                elif relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_DOOR_VERTICAL]

            elif tile_type == TILE_DOOR_VERTICAL:
                # Vertical doors can connect to vertical walls or corners above and below
                if relative_position == (-1, 0):  # Above
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_BOTTOM_LEFT, TILE_WALL_CORNER_BOTTOM_RIGHT]
                elif relative_position == (1, 0):  # Below
                    return neighbor_type in [TILE_WALL_VERTICAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_TOP_RIGHT]

            elif tile_type == TILE_DOOR_HORIZONTAL:
                # Horizontal doors can connect to horizontal walls or corners left and right
                if relative_position == (0, -1):  # Left
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_RIGHT, TILE_WALL_CORNER_BOTTOM_RIGHT]
                elif relative_position == (0, 1):  # Right
                    return neighbor_type in [TILE_WALL_HORIZONTAL, TILE_WALL_CORNER_TOP_LEFT, TILE_WALL_CORNER_BOTTOM_LEFT]

            return False  # If no rules apply, the neighbor is invalid



        def is_connected_to_door(y, x, tile_map, visited, score_map, original_direction=None):
            """
            Checks if a tile is connected to doors using Depth-First Search (DFS).
            Returns the count of doors this tile is connected to.
            """
            if (
                    (y, x) in visited
                    or not (0 <= x < self.width and 0 <= y < self.height)
                    or score_map[y][x] == -3
            ):
                return 0  # Changed from False to 0

            visited.add((y, x))

            if tile_map[y][x] in [TILE_DOOR_HORIZONTAL, TILE_DOOR_VERTICAL]:
                return 1  # Count this door

            door_count = 0  # Track total number of doors connected
            current_tile_type = tile_map[y][x]
            neighbors = []

            # Neighbor logic
            if current_tile_type == TILE_WALL_VERTICAL:
                neighbors = [(y + 1, x), (y - 1, x)]
            elif current_tile_type == TILE_WALL_HORIZONTAL:
                neighbors = [(y, x + 1), (y, x - 1)]
            elif current_tile_type == TILE_WALL_CORNER_TOP_LEFT:
                neighbors = [(y, x + 1), (y + 1, x)]
            elif current_tile_type == TILE_WALL_CORNER_TOP_RIGHT:
                neighbors = [(y, x - 1), (y + 1, x)]
            elif current_tile_type == TILE_WALL_CORNER_BOTTOM_LEFT:
                neighbors = [(y, x + 1), (y - 1, x)]
            elif current_tile_type == TILE_WALL_CORNER_BOTTOM_RIGHT:
                neighbors = [(y, x - 1), (y - 1, x)]
            elif current_tile_type in [TILE_DOOR_VERTICAL, TILE_DOOR_HORIZONTAL]:
                return 1
            else:
                neighbors = []

            # Check all valid neighbors and sum up their door connections
            for ny, nx in neighbors:
                if (ny, nx) != original_direction:  # Avoid checking tile we came from
                    if is_valid_tile_neighbor((y, x), (ny, nx), tile_map):
                        door_count += is_connected_to_door(
                            ny, nx, tile_map, visited.copy(), score_map, (y, x)
                        )

            return door_count



        for y in range(self.height):
            for x in range(self.width):
                if score_map[y][x] != -3:  # Only check valid tiles
                    door_count = is_connected_to_door(y, x, tile_map, set(), score_map)
                    if door_count > 0:  # If connected to at least one door
                        valid_wall_map[y, x] = door_count  # Could store the count instead of just 1

        return valid_wall_map



In [202]:
class LevelIndividual:
    """Represents an individual in the genetic algorithm."""
    def __init__(self, problem):
        self.problem = problem
        self.door_positions = problem.door_positions
        self.chromosome = self.generate_chromosome() # Chromosome is an N x N 2D tile matrix

    def generate_chromosome(self):
        # Step 1: Generate a random base chromosome
        chromosome = np.zeros((self.problem.height, self.problem.width), dtype=int)

        for y in range(self.problem.height):
            for x in range(self.problem.width):
                # Randomly assign any possible wall type
                chromosome[y, x] = random.choice([
                    TILE_WALL_VERTICAL,
                    TILE_WALL_HORIZONTAL,
                    TILE_WALL_CORNER_TOP_LEFT,
                    TILE_WALL_CORNER_TOP_RIGHT,
                    TILE_WALL_CORNER_BOTTOM_LEFT,
                    TILE_WALL_CORNER_BOTTOM_RIGHT,
                ])

        # Step 2: Add door positions into the chromosome
        for x, y, orientation in self.door_positions:
            chromosome[y, x] = orientation  # Set the door orientation at the given coordinates

        return chromosome


    def mutate(self, mutation_rate):
        """
        Mutate the chromosome by randomly changing tiles, excluding doors and validated tiles.
        """
        # Access the valid_wall_map from the problem
        _, _, valid_wall_map = self.problem.calculate_fitness(self.chromosome)

        # Create a list of all non-validated tiles
        non_validated_tiles = [
            (x, y) for y in range(self.problem.height) for x in range(self.problem.width)
            if valid_wall_map[y, x] == 0 and (x, y) not in [(door[0], door[1]) for door in self.problem.door_positions]
        ]

        # Calculate the number of mutations based on the mutation rate
        num_mutations = int(len(non_validated_tiles) * mutation_rate)

        # Randomly select tiles to mutate from the non-validated ones
        tiles_to_mutate = random.sample(non_validated_tiles, num_mutations)

        # Perform mutations on the selected tiles
        for x, y in tiles_to_mutate:
            self.chromosome[y, x] = random.choice([
                TILE_WALL_VERTICAL,
                TILE_WALL_HORIZONTAL,
                TILE_WALL_CORNER_BOTTOM_LEFT,
                TILE_WALL_CORNER_BOTTOM_RIGHT,
                TILE_WALL_CORNER_TOP_LEFT,
                TILE_WALL_CORNER_TOP_RIGHT
            ])


    def crossover(self, other_parent):
        """
        Perform crossover with another parent.
        Mix parts of the two maps.
        """
        child1 = deepcopy(self)
        child2 = deepcopy(other_parent)

        # Single-point crossover on the 2D matrix
        split_row = random.randint(0, self.problem.height - 1)

        child1.chromosome[:split_row, :] = self.chromosome[:split_row, :]
        child1.chromosome[split_row:, :] = other_parent.chromosome[split_row:, :]

        child2.chromosome[:split_row, :] = other_parent.chromosome[:split_row, :]
        child2.chromosome[split_row:, :] = self.chromosome[split_row:, :]

        return child1, child2


    def generate_display_map(self):
        """
        Generates a scored map from a chromosome, highlighting positive scores and
        using spaces for non-positive scores.

        Args:
            chromosome: The chromosome representing the map.

        Returns:
            str: Human-readable formatted display map as a string.
        """
        # Decode the map and calculate fitness and scores
        tile_map = self.chromosome
        fitness, score_map, _= self.problem.calculate_fitness(tile_map)

        # Create a display map based on scores (positive scores retain the original tile)
        scored_map = np.full_like(tile_map, " ", dtype=object)
        for y in range(self.problem.height):
            for x in range(self.problem.width):
                if score_map[y, x] > 0:  # Correctly check score_map with (y, x) indexing
                    scored_map[y, x] = tile_map[y, x]  # Assign tile for positive scores
                else:
                    scored_map[y, x] = " "  # Non-positive scores are blank spaces

        # Display logic to make it human-readable
        vertical = ' | '
        horizontal = '---'
        corner_bottom_left = ' |_'
        corner_bottom_right = '_| '
        corner_top_right = '‾| '
        corner_top_left = ' |‾'

        door_horizontal = 'HD'
        door_vertical = 'VD'

        output = ""
        for row in scored_map:
            formatted_row = []
            for tile in row:
                if tile == 0:  # Vertical wall
                    formatted_row.append(vertical.ljust(3))
                elif tile == 1:  # Horizontal wall
                    formatted_row.append(horizontal.ljust(3))
                elif tile == 2:  # Corner bottom-left
                    formatted_row.append(corner_bottom_left.ljust(3))
                elif tile == 3:  # Corner bottom-right
                    formatted_row.append(corner_bottom_right.ljust(3))
                elif tile == 4:  # Corner top-right
                    formatted_row.append(corner_top_right.ljust(3))
                elif tile == 5:  # Corner top-left
                    formatted_row.append(corner_top_left.ljust(3))
                elif tile == ' ':  # Empty space for non-positive scores
                    formatted_row.append('   ')
                elif tile == 7:  # Horizontal door
                    formatted_row.append(door_horizontal.ljust(3))
                elif tile == 8:  # Vertical door
                    formatted_row.append(door_vertical.ljust(3))
                else:  # Default case for unmapped tiles
                    formatted_row.append(str(tile).ljust(3))
            output += "[" + "".join(formatted_row) + "]\n"

        return output

In [203]:
params = parameters()

In [204]:
problem = LevelGenerationProblem(params)
# View the generated door positions
print("Door positions:", problem.door_positions)


Door positions: [(4, 5, 8), (3, 5, 8), (5, 4, 8), (4, 4, 8)]


In [205]:
ind1 = LevelIndividual(problem)

In [206]:
    # Decode the individual's chromosome into a map
decoded_map = ind1.chromosome
print("Decoded Map:")
print(decoded_map)


Decoded Map:
[[4 5 2 2 4 0 2 3 1 2]
 [4 2 0 1 0 5 2 2 5 3]
 [1 5 2 5 0 4 1 5 2 3]
 [4 1 0 2 0 0 1 3 0 4]
 [2 1 5 2 8 8 4 3 3 5]
 [1 2 5 8 8 5 0 5 2 2]
 [2 1 4 4 1 5 1 4 2 1]
 [2 0 3 0 1 0 1 1 3 0]
 [4 1 3 0 1 2 2 3 2 0]
 [5 0 5 1 3 1 1 3 3 3]]


In [207]:
# Calculate fitness
fitness, scored_map, validated_wall_map = problem.calculate_fitness(ind1.chromosome)
print("Fitness:", fitness)
print("Scored Map:", scored_map)


Fitness: 18
Scored Map: [[ -3  -3  -3  -3  -3  -3  -3  -3  -3  -3]
 [ -3  -3  -3  -3  20  -3  -3  -3  -3  -3]
 [ -3  -3  -3  -3  20  -3  -3  -3  10  -3]
 [ -3  -3  -3  -3  60  30  -3  -3  -3  -3]
 [ -3  -3  -3  -3 160  -3  -3  -3  -3  -3]
 [ -3  -3  -3  -3  -3  -3  -3  -3  -3  -3]
 [ -3  10  10  -3  -3  10  10  -3  -3  -3]
 [ -3  -3  -3  10  -3  10  -3  10  -3  -3]
 [ -3  -3  -3  -3  -3  -3  -3  -3  -3  10]
 [ -3  -3  -3  10  -3  -3  10  -3  -3  -3]]


In [208]:
scored_map = ind1.generate_display_map()
print(scored_map)


[                              ]
[             |                ]
[             |           |_   ]
[             |  |             ]
[            VD                ]
[                              ]
[   ---‾|        |‾---         ]
[          |     |    ---      ]
[                            | ]
[         ---      ---         ]



In [209]:
print("Valid wall Map:\n", validated_wall_map)

Valid wall Map:
 [[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]


In [210]:
def choose_parents(population):
    # Tournament selection
    tournament_size = 5
    tournament = random.sample(population, tournament_size)
    return max(tournament, key=lambda x: x.cost)

In [211]:
# %%
def run_genetic(prob, params):
    # Read Variables
    population_size = params.population_size
    rate_of_gene_mutation = params.mutation_rate
    # cost_function = prob.calculate_fitness()
    number_of_children_per_generation = int(params.birth_rate_per_generation * population_size)
    max_number_of_generations = params.max_number_of_generations

    # Create Our Population
    population = []
    best_solution = LevelIndividual(prob)  # Use LevelIndividual
    best_solution.cost = -100000  # Initialize with a very low cost

    for i in range(population_size):
        new_individual = LevelIndividual(prob)  # updated individual type
        new_individual.cost, _, _ = prob.calculate_fitness(new_individual.chromosome)  # removed score map
        if new_individual.cost > best_solution.cost:
            best_solution = deepcopy(new_individual)
        population.append(new_individual)

    # Start Loop
    for i in range(max_number_of_generations):
        # Start generation loop
        children = []
        while (len(children) < number_of_children_per_generation):
            # choose Parents
            parent1 = choose_parents(population)
            parent2 = choose_parents(population)

            # Create children
            child1, child2 = parent1.crossover(parent2)
            child1.mutate(rate_of_gene_mutation)
            child2.mutate(rate_of_gene_mutation)

            child1.cost, _ , _= prob.calculate_fitness(child1.chromosome)
            child2.cost, _ , _= prob.calculate_fitness(child2.chromosome)

            # add children to population
            children.append(child1)
            children.append(child2)

        # add children
        population += children

        # sort population
        population = sorted(population, key=lambda x: x.cost, reverse=True)  # Sort in descending order

        # cull population
        population = population[:population_size]

        # check solution
        if population[0].cost > best_solution.cost:
            best_solution = deepcopy(population[0])

        print(f"Generation {i}: Best solution cost = '{best_solution.cost}'")

    print( "Best solution cost = '{best_solution.cost}'", best_solution.cost)
    return (population, best_solution)


In [212]:

params1 = parameters()
problem1 = LevelGenerationProblem(params1)

In [213]:
pop, best = run_genetic(problem1,params1)

Generation 0: Best solution cost = '185'
Generation 1: Best solution cost = '236'
Generation 2: Best solution cost = '306'
Generation 3: Best solution cost = '417'
Generation 4: Best solution cost = '417'
Generation 5: Best solution cost = '423'
Generation 6: Best solution cost = '541'
Generation 7: Best solution cost = '560'
Generation 8: Best solution cost = '567'
Generation 9: Best solution cost = '645'
Generation 10: Best solution cost = '645'
Generation 11: Best solution cost = '645'
Generation 12: Best solution cost = '645'
Generation 13: Best solution cost = '658'
Generation 14: Best solution cost = '736'
Generation 15: Best solution cost = '788'
Generation 16: Best solution cost = '788'
Generation 17: Best solution cost = '814'
Generation 18: Best solution cost = '840'
Generation 19: Best solution cost = '840'
Generation 20: Best solution cost = '905'
Generation 21: Best solution cost = '905'
Generation 22: Best solution cost = '918'
Generation 23: Best solution cost = '944'
Ge

In [214]:
fitness, scored_map, validated_wall_map = problem.calculate_fitness(best.chromosome)
print("Fitness:", fitness)
print("Scored Map:\n", scored_map)

Fitness: 1165
Scored Map:
 [[ -3  40  40  40  40  10  10  20  20  20]
 [ -3  40  40  40  40  10  10  20  -3  20]
 [ 40  40  -3  40 120 120  60  20  20  20]
 [ 40  40  -3  40 120 120 120  20  20  20]
 [ 40  40  40  40  -3  -3  40  20  -3  20]
 [ 40  40  40  40  -3  10  40  20  -3  20]
 [ 40  40  40  40  -3  10 120  60  20  20]
 [ 40  40  40  40  -3  -3 120 120  -3  -3]
 [ 40  40  40  40  40  40 120 120  40  40]
 [ 40  40  40  40  40  40  40  40  40  40]]


In [215]:
decoded_map = best.chromosome
print("Decoded Map:\n",decoded_map)

Decoded Map:
 [[3 5 1 1 4 5 4 5 1 4]
 [5 2 1 4 0 2 3 0 0 0]
 [5 4 5 0 2 7 1 3 2 3]
 [0 0 2 2 1 7 4 5 1 4]
 [0 2 1 4 2 5 0 0 2 0]
 [2 1 4 0 0 0 0 0 0 0]
 [5 1 3 0 3 0 0 0 2 3]
 [0 5 4 0 4 0 8 8 0 4]
 [0 0 0 2 1 1 3 2 1 4]
 [2 3 2 1 1 1 1 1 1 3]]


In [216]:
display_map = best.generate_display_map()
print(display_map)

[    |‾------‾|  |‾‾|  |‾---‾| ]
[    |_---‾|  |  |__|  |     | ]
[ |‾‾|     |  |_HD ---_|  |__| ]
[ |  |     |_---HD ‾|  |‾---‾| ]
[ |  |_---‾|        |  |     | ]
[ |_---‾|  |     |  |  |     | ]
[ |‾---_|  |     |  |  |  |__| ]
[ |  |‾‾|  |       VD VD       ]
[ |  |  |  |_------_|  |_---‾| ]
[ |__|  |_------------------_| ]



In [217]:
print("Valid wall Map:\n",validated_wall_map)

Valid wall Map:
 [[0 2 2 2 2 0 0 1 1 1]
 [0 2 2 2 2 0 0 1 0 1]
 [2 2 0 2 2 1 1 1 1 1]
 [2 2 0 2 2 1 2 1 1 1]
 [2 2 2 2 0 0 2 1 0 1]
 [2 2 2 2 0 0 2 1 0 1]
 [2 2 2 2 0 0 2 1 1 1]
 [2 2 2 2 0 0 1 1 0 0]
 [2 2 2 2 2 2 2 2 2 2]
 [2 2 2 2 2 2 2 2 2 2]]
